# I. 반도체 박막 데이터 탐색적 분석(EDA)

---
### 1) 데이터 로드 : dataframe (pandas module 사용)  
https://dacon.io/competitions/official/235554/overview/
train과 test가 별도로 나누어 지지 않았고, train.csv 하나만 있다.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc
gc.collect() # garbage collector : 메모리 관리

In [ ]:
import os

---
전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'
RAW_DATA_FILE = 'train.csv'
RAW_DATA_PATH = os.path.join(DATA_DIR, RAW_DATA_FILE)

In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv(RAW_DATA_PATH)

---
### 2) 데이터 기본 정보 확인

In [ ]:
raw_df.info()

In [ ]:
raw_df.columns

In [ ]:
raw_df.describe()

In [ ]:
'''
# 아래 방법으로 데이터프레임의 프로파일을 보는 방법이 있으나, 처리 시간이 매우 많이 걸린다.
# import pandas_profiling
# pr =  raw_df.profile_report() # 바로 결과 보기
# pr.to_file('./data/pr_report.html') # pr_report.html 파일로 저장 
# pr.to_widgets()


# 아래와 같이 minimal=True로 하면 처리가 조금 일찍 끝난다.
import pandas_profiling as pp
pr = pp.ProfileReport(raw_df, minimal=True)
pr.to_file('./data/pr_report.html') # pr_report.html 파일로 저장 
'''

In [ ]:
# 결측 데이터 확인
raw_df.isnull().sum()[raw_df.isnull().sum().values > 0]

In [ ]:
# layer1 ~ 4만 분리해서 확인
# 질화규소(layer_1)/이산화규소(layer_2)/질화규소(layer_3)/이산화규소(layer_4)
raw_df.iloc[:,:4]

In [ ]:
# 4개의 레이어가 모두 동일하게 각각 10을 단위로 10~ 300까지 30가지의 경우수를 가짐.
# 그래서 train_df의 크기가 81만 row를 가짐.(30 * 30 * 30 * 30)
print(raw_df['layer_1'].unique())
print(raw_df['layer_2'].unique())
print(raw_df['layer_3'].unique())
print(raw_df['layer_4'].unique())

In [ ]:
30**4

In [ ]:
# 0 ~ 225 파장번호에 따른 값 확인
raw_df.iloc[:,4:]

In [ ]:
# 0번째 파장번호의 반사율값을 확인.
wave_0_df = raw_df.iloc[:,4:5]
wave_0_df

---
### 3) 데이터 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 각 레이어 별 두께와 파장 값에 따른 평균값 행렬을 구한다. 
train_layer1 = raw_df.drop(['layer_2', 'layer_3', 'layer_4'], axis=1).groupby(['layer_1']).mean()
train_layer2 = raw_df.drop(['layer_1', 'layer_3', 'layer_4'], axis=1).groupby(['layer_2']).mean()
train_layer3 = raw_df.drop(['layer_1', 'layer_2', 'layer_4'], axis=1).groupby(['layer_3']).mean()
train_layer4 = raw_df.drop(['layer_1', 'layer_2', 'layer_3'], axis=1).groupby(['layer_4']).mean()

In [ ]:
train_layer1

In [ ]:
train_layer1.T

In [ ]:
figure, ((ax1, ax2, ax3, ax4)) = plt.subplots(nrows=4, ncols=1)
figure.set_size_inches(15, 30)

sns.heatmap(train_layer1.T, ax=ax1)
ax1.set_title('layer_1 heatmap',fontsize=10)
sns.heatmap(train_layer2.T, ax=ax2)
ax2.set_title('layer_2 heatmap', fontsize=10)
sns.heatmap(train_layer3.T, ax=ax3)
ax3.set_title('layer_3 heatmap', fontsize=10)
sns.heatmap(train_layer4.T, ax=ax4)
ax4.set_title('layer_4 heatmap', fontsize=10)

In [ ]:
# 히스토그램 분포 그려보기. : wave_0 파장 번호 0에 해당하는 데이터
# wave_0은 0에서 0.8 사이의 굴절율을 가짐.
sns.distplot(wave_0_df, kde=True) # 커널밀도
plt.show()

In [ ]:
# 샘플 데이터 추출
# layer1에 대한 정보만 우선 추출.
# layer1,2,3,4의 조합이여서 같은 두께에 대해 여러 값들이 존재함. 
# 같은 두께에 대해 하나로 집계할 필요성이 있음.
layer1_sample_df = raw_df[['layer_1','0','1','2','3','4','5','6','7','8','9','10']] # , '220', '221', '222','223','224','225']
layer1_sample_df.head()

In [ ]:
# layer1의 두께별로 반사율값을 평균치 집계
sample_df_agg = layer1_sample_df.groupby('layer_1').agg('mean')
sample_df_agg.head()

In [ ]:
# 위에서 집게한 값을 히트맵으로 시각화해봄

# x : 파장넘버, y: 박막 두께.
# 파장 0,1에 대해layer_1의 박막두께에 따른 평균적인 반사율값.
# 박막두께가 100, 300 에서 높은 반사율. 10, 200에서는 낮은 반사율을 보여줌.
plt.rcParams["figure.figsize"] = (5,10)
sns.heatmap(sample_df_agg, cmap='YlGnBu') # , ax=ax)
plt.title('raw_df heatmap', fontsize=20)
plt.show()

In [ ]:
# 220~225
layer1_sample_df_tail = raw_df[['layer_1','220', '221', '222','223','224','225']]
layer1_sample_df_tail_agg = layer1_sample_df_tail.groupby('layer_1').agg('mean')
# sample_df_agg.head()

In [ ]:
plt.rcParams["figure.figsize"] = (5,10)
sns.heatmap(layer1_sample_df_tail_agg, cmap='YlGnBu') # , ax=ax)
plt.title('train_df heatmap', fontsize=20)
plt.show()
# 파장이 클수록 반사율이 높게나옴.
# 레이어 두께가 70을 간격으로 보여짐.

# 여기까지 결론은, 두께가 얇으면 투과율이 높아서 반사율이 상대적으로 낮고, 두께가 크면 반사율이 높다는 특징을 확인.

In [ ]:
layer1_sample_df.shape

In [ ]:
# 이제부터는 모델을 돌리기 위해 데이터를 변환하는 과정을 거침.
# 멜팅 : 데이터를 재구조화.
# layer1에 대해서 우선 멜팅. 야래 결과를 보면 2차원 행렬로 되어 있는 반사율 데이터가 하나의 컬럼 데이터로 직렬화 됨.
layer1_melt_df = pd.melt(layer1_sample_df, id_vars=['layer_1'],
                        var_name='wave', value_name='reflect_ratio')
layer1_melt_df

In [ ]:
# 첫 번째 행의 데이터 중 반사율 값만 선택, 그래프를 그리기 위해서...
train_1 = raw_df.iloc[0:1, 5:]
train_1

In [ ]:
train_1_t = train_1.transpose()
train_1_t.plot(figsize=(25,5),grid=True)

In [ ]:
train_1_t_r = train_1_t[::-1] # 순서를 거꾸로 바꾸기
train_1_t_r.plot(figsize=(25,5),grid=True)

In [ ]:
# 파장 값 역수(reciprocal)
train_1_t_rec = 1/train_1_t
train_1_t_rec.plot(figsize=(25,5),grid=True)

In [ ]:
# 파장 값 역수(reciprocal), 순서를 거꾸로
train_1_t_rec = 1/train_1_t_r
train_1_t_rec.plot(figsize=(25,5),grid=True)

In [ ]:
train_2 = raw_df[(raw_df['layer_1']==10) & (raw_df['layer_2']==10) & (raw_df['layer_3']==170) & (raw_df['layer_4']==90)]
train_2

In [ ]:
train_2 = train_2.iloc[0:1, 5:]
train_2

In [ ]:
train_2_t = train_2.transpose()
train_2_t.plot(figsize=(25,5),grid=True)

In [ ]:
train_2_t = train_2_t[::-1] # 순서를 거꾸로 바꾸기
train_2_t.plot(figsize=(25,5),grid=True)

In [ ]:
raw_df